# ADALA Quickstart

In this notebook, we are going to run through some of the common tasks for creating data labeling agents with ADALA. In this example, we're going to create a data labeling agent for a text classification task - labeling our text samples as either "Subjective or "Objective" statements. 

This agent will be LLM-based, so we will use [OpenAI's API](https://platform.openai.com/). You will to generate an API key and set it as an environment variable as follows: 

```
export OPENAI_API_KEY=your_openai_api_key
```

Now, let's begin. 

## Dataset Creation
First, let's use a dataset of product reviews stored in pandas dataframe. This will help us manage our data as we add more attributes, like predictions and labels for subjectivity and objectivity over time. 

In [1]:
import pandas as pd

df = pd.DataFrame([
    ["The mic is great.", "Subjective"],
    ["Will order from them again!", "Subjective"],
    ["Not loud enough and doesn't turn on like it should.", "Objective"],
    ["The phone doesn't seem to accept anything except CBR mp3s", "Objective"],
    ["All three broke within two months of use.", "Objective"]
], columns=["text", "ground_truth"])

df

,text,ground_truth
0,The mic is great.,Subjective
1,Will order from them again!,Subjective
2,Not loud enough and doesn't turn on like it sh...,Objective
3,The phone doesn't seem to accept anything exce...,Objective
4,All three broke within two months of use.,Objective


We instantiate Dataset that uses this pandas dataframe as a data source. Dataset object takes care of input data schema and data streaming:

## Create Agent

To create Agent, we need to to define 2 things:

**Skills** - Agent's abilities are defined as _Skills_. Each agent can possess many different skills. In our case, this agent only has one labeling skill, to produce a classification of Subjective or Objective for a given piece of text.  To define this skill, we will leverage an LLM, passing it instructions and the set of labeles we expect to receive back. 

**Environment** - that is where the Agent receives ground truth signal to improve its skill. Since we already created ground truth dataset, we can simply refer to the column from the dataframe. In the real world scenario, you may consider using a different environment where ground truth signal can be obtained asynchoronously by gathering real human feedback during agent's learning phase.

In [4]:
from adala.agents import Agent
from adala.environments import StaticEnvironment
from adala.skills import ClassificationSkill
from adala.runtimes import OpenAIChatRuntime, GuidanceRuntime
from rich import print


agent = Agent(
    # define the agent's labeling skill that should classify text onto 2 categories
    skills=ClassificationSkill(
        name='subjectivity_detection',
        description='Understanding subjective and objective statements from text.',
        instructions='Classify a product review as either expressing "Subjective" or "Objective" statements.',
        labels={'prediction': ['Subjective', 'Objective']},
        input_template='Input: {text}',
        output_template='Output: {prediction}'
    ),
    
    # basic environment extracts ground truth signal from the input records
    environment=StaticEnvironment(
        df=df,
        ground_truth_columns={'prediction': 'ground_truth'}
    ),
    
    runtimes = {
        'openai': GuidanceRuntime(),
    },
    default_runtime='openai',
    
    teacher_runtimes = {
      'openai-gpt3': OpenAIChatRuntime(model='gpt-3.5-turbo'),
      'openai-gpt4': OpenAIChatRuntime(model='gpt-4'),
    },
    
    # NOTE! If you don't have an access to gpt4 - replace it with "openai-gpt3"
    default_teacher_runtime='openai-gpt4'
)

print(agent)

Agent Instance

Environment: StaticEnvironment
Skills: subjectivity_detection
Runtimes: openai
Default Runtime: openai
Default Teacher Runtime: openai-gpt4

## Learning Agent

We will now let Agent learn from the ground truth. After every action, Agent returns its _Experience_, where it stores various observations like predicted data, errors, accuracy, etc.

In [5]:
ground_truth_signal = agent.learn(learning_iterations=3, accuracy_threshold=0.95)

=> Iteration #0: Getting feedback, analyzing and improving ...

Applying skill: subjectivity_detection

100%|█| 5/5 [00:01<00:00, 


Predictions and feedback:

 prediction__fb                             text                                       ground_truth   prediction  
 ───────────────────────────────────────────────────────────────────────────────────────────────────────────────── 
  Prediction is correct.                     The mic is great.                          Subjective     Subjective  
  Prediction is correct.                     Will order from them again!                Subjective     Subjective  
  Prediction is incorrect. Correct answer:   Not loud enough and doesn't turn on like   Objective      Subjective  
  "Objective"                                it should.                                                            
  Prediction is correct.                     The phone doesn't seem to accept           Objective      Objective   
                                             anything except CBR mp3s                                              
  Prediction is correct.                     All three broke within two months of       Objective      Objective   
                                             use.

...

"{assistant}":
Sure, I’d be happy to help you with this prompt engineering problem. Please provide me with the current prompt and 
the examples with user feedback.

"{user}":

## Current prompt
"Analyze the provided product review and ascertain whether it predominantly expresses 'Subjective' or 'Objective' 
statements. Classify it as 'Subjective' if the review primarily discloses personal feelings, emotions, or 
preferences. However, should the review describe factual details about the product’s functionality or performance, 
even if these details stem from personal experiences, yet do not assert personal emotions or preferences, classify 
it as 'Objective'. Keep in mind that a review can draw from personal experiences yet still remain 'Objective' if it
imparts information about the product's performance or functionality without declaring personal emotions or 
preferences."

## Examples
### Example #0

Input: The mic is great.

Output: Subjective

User feedback: Prediction is correct.


### Example #1

Input: Will order from them again!

Output: Subjective

User feedback: Prediction is correct.


### Example #2

Input: Not loud enough and doesn't turn on like it should.

Output: Subjective

User feedback: Prediction is incorrect. Correct answer: "Objective"


### Example #3

Input: The phone doesn't seem to accept anything except CBR mp3s

Output: Objective

User feedback: Prediction is correct.


### Example #4

Input: All three broke within two months of use.

Output: Objective

User feedback: Prediction is correct.



Summarize your analysis about incorrect predictions and suggest changes to the prompt.

"{assistant}":
## Analysis 

Looking at the examples provided, it appears that the model mostly predicts correctly, except for Example #2. 

For Example #2, the user feedback suggests that the model's prediction is incorrect. The model categorized it as 
'Subjective' when it should be 'Objective'. The user comment "Not loud enough and doesn't turn on like it should" 
is a factual statement based on the user's experiences with the product. This reflects the fact that the product 
does not operate as it should, according to its specifications or expected performance, making it an 'Objective' 
statement.

I believe that this discrepancy might be due to the negative aspect of this review. The model could have 
potentially interpreted the negative sentiment as a sign of subjectivity.

## Prompt Suggestion 

Based on the analysis, we can refine the prompt by adding more clarity around subjective and objective attitudes, 
both positive and negative. Negative criticism that is based on factual aspects of a product or service should also
be seen as 'Objective'. We could possibly add a line emphasizing this in the prompt. 

Here's a suggestion:

"Analyze the provided product review and ascertain whether it predominantly expresses 'Subjective' or 'Objective' 
statements. Classify it as 'Subjective' if the review primarily discloses personal feelings, emotions, or 
preferences, regardless of them being positive or negative. However, should the review describe factual details 
about the product’s functionality or performance, even if these details reveal issues or negative aspects of the 
product, classify it as 'Objective'. A review can draw from personal experiences yet still remain 'Objective' if it
imparts information about the product's performance or functionality without declaring personal emotions or 
preferences. Pay attention to whether the statements hold true irrespective of who is using the product."

"{user}":

Now please carefully review your reasoning in Step 1 and help with Step 2: refining the prompt.

## Current prompt
"Analyze the provided product review and ascertain whether it predominantly expresses 'Subjective' or 'Objective' 
statements. Classify it as 'Subjective' if the review primarily discloses personal feelings, emotions, or 
preferences. However, should the review describe factual details about the product’s functionality or performance, 
even if these details stem from personal experiences, yet do not assert personal emotions or preferences, classify 
it as 'Objective'. Keep in mind that a review can draw from personal experiences yet still remain 'Objective' if it
imparts information about the product's performance or functionality without declaring personal emotions or 
preferences."

## Follow this guidance to refine the prompt:

1. The new prompt should should describe the task precisely, and address the points raised in the user feedback.
    
2. The new prompt should be similar to the current instruction, and only differ in the parts that address the 
issues you identified in Step 1.
    Example:
    - Current prompt: "The model should generate a summary of the input text."
    - New prompt: "The model should generate a summary of the input text. Pay attention to the original style."
        
3. Reply only with the new prompt. Do not include input and output templates in the prompt.

Train is done!

Let's see the final instructions:

In [6]:
print(agent.skills)

Total Agent Skills: 1

subjectivity_detection
"Analyze the provided product review and classify it as 'Subjective' or 'Objective'. If the review primarily shares
personal feelings, emotions, or preferences, label it as 'Subjective'. Subjectivity may be presented through both 
positive and negative attitudes. If the review, on the other hand, conveys factual details or highlights positive 
or negative attributes about the product's functionality or performance, classify it as 'Objective'. These factual 
details may stem from personal experiences but do not express personal emotions or preferences. Remember, a review 
may contain personal experiences but remain 'Objective' if it imparts information about the product's performance 
or functionality without expressing personal feelings, preferences, or assertions. Objective reviews present 
statements that would be generally true, regardless of the user."

... and predictions created by the skill:

In [8]:
agent.run()

Applying skill: subjectivity_detection

100%|█| 5/5 [00:00<00:00, 


,text,ground_truth,prediction
0,The mic is great.,Subjective,Subjective
1,Will order from them again!,Subjective,Subjective
2,Not loud enough and doesn't turn on like it sh...,Objective,Subjective
3,The phone doesn't seem to accept anything exce...,Objective,Objective
4,All three broke within two months of use.,Objective,Objective


## Applying learned skills to the real data

Now as we have our Agent with evolved "subjectivity detection" skill, we can apply it to the real dataset without ground truth data:

In [9]:
test_df = pd.DataFrame([
    "Doesn't hold charge.",
    "Excellent bluetooth headset",
    "I love this thing!",
    "VERY DISAPPOINTED."
], columns=['text'])
test_df

,text
0,Doesn't hold charge.
1,Excellent bluetooth headset
2,I love this thing!
3,VERY DISAPPOINTED.


In [10]:
predictions = agent.run(test_df)

Applying skill: subjectivity_detection

100%|█| 4/4 [00:01<00:00, 


In [11]:
predictions

,text,prediction
0,Doesn't hold charge.,Objective
1,Excellent bluetooth headset,Objective
2,I love this thing!,Subjective
3,VERY DISAPPOINTED.,Subjective
